# Signals Boosting

NOTE: This notebook depends upon the the Retrotech dataset. If you have any issues, please rerun the [Setting up the Retrotech Dataset](1.setting-up-the-retrotech-dataset.ipynb) notebook.

In [ ]:
#%load <shared py file> product_search_request
import sys
sys.path.append('..')
from aips import *  
from pyspark.sql import SparkSession
engine = get_engine()
spark = SparkSession.builder.appName("AIPS").getOrCreate()
products_collection = engine.get_collection("products")

## Keyword Search with No Signals Boosting

### Listing 4.5

In [ ]:
query = "ipad"
request = product_search_request(query)
response = products_collection.search(request)
display_product_search(query, engine.docs_from_response(response))

## Create Signals Boosts (Signals Aggregation)

### Listing 4.6

In [ ]:
signals_collection = engine.get_collection("signals")

signals_boosting_collection = engine.create_collection("signals_boosting")
print("Aggregating Signals to Create Signals Boosts...")
create_view(signals_collection, "signals")

signals_aggregation_query = """
SELECT q.target AS query, c.target AS doc, COUNT(c.target) AS boost
FROM signals c LEFT JOIN signals q ON c.query_id = q.query_id
WHERE c.type = 'click' AND q.type = 'query'
GROUP BY query, doc
ORDER BY boost DESC
"""

dataframe = spark.sql(signals_aggregation_query)
signals_boosting_collection.write_from_dataframe(dataframe)
print("Signals Aggregation Completed!")

## Search with Signals Boosts Applied

### Listing 4.7

In [ ]:
query = "ipad"
signals_boosts_query = {
    "query": query,
    "fields": ["doc", "boost"],
    "limit": 10,
    "params": {
        "defType": "edismax",
        "qf": "query",
        "sort": "boost desc"
    }
}
response = signals_boosting_collection.search(signals_boosts_query)
boosts_docs = engine.docs_from_response(response)
print(f"Boost Documents: \n{boosts_docs}")
product_boosts = " ".join([f'"{entry["doc"]}"^{str(entry["boost"])}' 
                           for entry in boosts_docs])
print(f"\nBoost Query: \n{product_boosts}")

request = product_search_request(query)
request["params"]["boost"] = "sum(1,query({! df=upc v=$signals_boosting}))"
request["params"]["signals_boosting"] = product_boosts
response = products_collection.search(request)
display_product_search(query, (response))

## Success!

You have now implemented your first AI-powered search algorithm: Signals Boosting. This is an overly simplistic implementation (we'll dive much deeper into signals boosting improvements in chapter 8), but it demonstrates the power of leveraging reflected intelligence quite well. We will dive into other Reflected Intelligence techniques in future chapters, such as Collaborative Filtering (in chapter 9 - Personalized Search) and Machine-learned Ranking (in chapter 10 - Learning to Rank).

Up next: Chapter 5 - [Knowledge Graph Learning](../ch05/1.open-information-extraction.ipynb)

